# Notebook example for HRTEM simulation postprocessing
This notebook illustrates how to use HyperSpy to analyse HRTEM simulation results produced. The results should already be converted to a hyperspy-readable format, such as ".hspy" and preferably through the use of `mul2py`s functionality. 

## Content
  1. [Loading and inspection](#1-loading-and-inspecting-your-results)
  3. [Making thickness profiles](3-making-thickness-profiles-of-atomic-column-scattering)
  4. [Other options (development)](4-other-options)

## 1 Loading and inspecting your results
Start by setting up matplotlib and importing required packages:

In [1]:
%matplotlib qt
import hyperspy.api as hs

Next, we load the result into a hyperspy signal:

In [5]:
from pathlib import Path
data_path = Path(r"C:\Users\emilc\OneDrive - NTNU\MULTEM_Results\HRTEM_results.hspy")
#data_path = Path("HRTEM_results.hspy")
signal = hs.load(str(data_path))

It is always useful to print some information about the signal and its axes:

In [6]:
print(signal.metadata)
print(signal.axes_manager)

├── General
│   ├── elapsed_time = 8.357123999997974
│   ├── original_filename = HRTEM_results.ecmat
│   └── title = HRTEM_results
├── Signal
│   ├── binned = False
│   └── signal_type = 
└── SimulationParameters
    ├── E_0 = 200.0
    ├── nx = 2048.0
    ├── ny = 2048.0
    ├── obj_lens_c_10 = -8.958454761414817
    ├── obj_lens_c_30 = 0.00032
    ├── spec_atoms = array([[13.    , 13.    , 13.    , ..., 13.    , 13.    , 13.    ],
       [ 1 ...  0.    ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]])
    ├── spec_dz = 2.025
    ├── spec_lx = 40.5
    ├── spec_ly = 40.5
    ├── spec_lz = 81.0
    ├── thick = array([ 0.   ,  2.025,  4.05 ,  6.075,  8.1  , 10.125, 12.15 , 14.175,
        ... 2.775,
       64.8  , 66.825, 68.85 , 70.875, 72.9  , 74.925, 76.95 , 78.975])
    └── thick_type = 2.0

<Axes manager, axes: (40|2048, 2048)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
    

If the file was generated by `mul2py.buildtools.builders.make_signal()`, the metadata should be useful and relevant to the simulation type, and the axes should be calibrated. Here, we can see that the simulation was performed with $ E=200 $ kV, the potentials sampling was $ 2048\times2048 $, and we can also see the specimen information. From the axes manager, we can see there are three dimensions, namely the thickness (` "z" `), and the $x$ and $y$ dimensions (` "x" `, and ` "y" `). 

Next, we plot the signal to take a look at these axes. 

In [7]:
signal.plot() #Plot the signal


We see that the total intensity of the images changes in a non-linear way, which we expect from this phase-contrast technique. The atom-sites start out as small dark circles that gradually expand, making the last image to appear like an image of bright points on a dark background. In other words, the bright spots are _not_ the atom-sites but rather the gap between them!. To invest this contrast reversial in more detail, you can show the intensity within a certain small region as a function of thickness by making thickness-profiles:

## 2 Making contrast profiles
If we want to make a thickness profile of the phase contrast from an atomic site, we must first make a region of interest and use it to extract the column from the signal:

In [8]:
roi = hs.roi.CircleROI(cx=19.07, cy=17.13, r=1.025) #Make a region of interest
print(roi)
roi.add_widget(signal, axes=['x', 'y']) #Connect the roi to the signal
cropped_signal = roi(signal) #Extract the roi from the signal (does not affect the original signal)


CircleROI(cx=19.07, cy=17.13, r=1.025)


With this cropped signal, we can integrate the signal space to take a look at the "collective" phase changes from this atomic column and how it develops through the thickness:

In [9]:
scattering_thickness_profile = cropped_signal.sum(axis=('x', 'y')) #Sum the signal in the x and y axes

scattering_thickness_profile.plot() #Plot the thickness profile of the phase-contrast


The above cell makes a thickness profile for the complete thickness. If you only want a every other thickness for instance, you can instead call:

In [10]:
scattering_thickness_profile.inav[0:-1:2].plot()

## 4 Other options
Other options are also possible. For instance, you can use machine learning algorithms to attempt to extract interference behaviour as a function of thickness, and/or as a function of defocus (if you have made several different defocus simulations). This is not presently covered in this guide, but may be added if there is interest.
